In [1]:
import requests
from io import BytesIO
from PIL import Image
import os

from tqdm.auto import tqdm
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

import torch
import torchvision
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset

from DataHandlers import ImageDataset, InMemDataLoader

/opt/homebrew/Caskroom/miniconda/base/envs/ml/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/homebrew/Caskroom/miniconda/base/envs/ml/lib/python3.9/site-packages/torchvision/image.so, 0x0006): symbol not found in flat namespace '__ZN3c106detail19maybe_wrap_dim_slowIxEET_S2_S2_b''If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [30]:
class ThresholdModel():
    def __init__(self):
        pass
    
    def train(self, loader):
        pass
    
    def predict(self, loader):
        for batch in loader:
            images1, images2, ids = batch
            delta = ((images1 - images2) ** 2).mean(dim=(1,2,3))
            pred = (delta < 0.02) * 1
            yield pred

class Wrapper():
    def __init__(
                self, 
                model, 
                paths = {'train': 'data/train.csv', 'valid': 'data/valid.csv', 'test': 'data/test.csv'},
                sample_sizes = {'train': 100, 'valid': 100, 'test': 100},
                transform = None,
                inMemLoaders = False
            ):
        
        self.model = model
        self.paths = paths
        self.sample_sizes = sample_sizes
        
        def _create_metadata(dataset_type):
            if dataset_type == 'train':
                return pd.read_csv(paths['train'])[:sample_sizes['train']]
            elif dataset_type == 'valid':
                return pd.read_csv(paths['valid'])[:sample_sizes['valid']]
            elif dataset_type == 'test':
                return pd.read_csv(paths['test'])[:sample_sizes['test']]
            else:
                raise ValueError(f"Unknown dataset type: {dataset_type}")

        def _create_dataloader(dataset, batch_size, shuffle):
            if inMemLoaders:
                return InMemDataLoader(
                    dataset,
                    batch_size=batch_size,
                    shuffle=shuffle,
                )
            else:
                return DataLoader(
                    dataset,
                    batch_size=batch_size,
                    shuffle=shuffle,
                )
        
        if transform is None:
            transform = T.Compose([
                T.Resize(200),
                T.CenterCrop(200),
                T.ToTensor(),
                T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])

        dataset_types = ['train', 'valid', 'test']

        self.datasets = {
            dataset_type: ImageDataset(_create_metadata(dataset_type), transform=transform)
            for dataset_type in dataset_types
        }

        self.loaders = {
            dataset_type: _create_dataloader(dataset, batch_size=20, shuffle=(dataset_type=='train'))
            for dataset_type, dataset in self.datasets.items()
        }
        
        
    def train(self):
        self.model.train(self.loaders['train'])
        
    def accuracy(self):
        train_preds = self.model.predict(self.loaders['valid'])
        
        n, true_preds = 0, 0
        for (_, _, truth), pred in zip(self.loaders['valid'], train_preds):
            true_preds += len((truth == pred).nonzero())
            n += len(pred)
            
        print(f'Validation accuracy: {true_preds / n}')
        
    def save_test_preds(self, path='res.csv'):
        ids = []
        preds = []

        for batch in self.loaders['test']:
            ids.extend(batch[2])
            
        for p in self.model.predict(self.loaders['test']):
            preds.extend(p)
        
        all_ids = pd.DataFrame({
            'ID': range(2, len(self.loaders['test'])),
        })
        res = pd.DataFrame({
            'ID': [obj.item() for obj in ids],
            'is_same': [obj.item() for obj in preds]
        }).drop_duplicates()

        res = all_ids.merge(res, on='ID', how='left').fillna(0)
        res.to_csv(path, index=False)
        print(f'Saved test predictions to {path}')
        
model = ThresholdModel()
wrapper = Wrapper(model)
wrapper.train()
wrapper.accuracy()
wrapper.save_test_preds()